In [1]:
import pandas as pd 
import numpy as np 

In [2]:
trip_file=pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\trips_india.csv")
trip_file.head()

,trip_id,assignment_id,driver_id,vehicle_id,route_id,start_time,end_time,distance_km,duration_min,fuel_before_l,fuel_after_l,fuel_used_l,co2_kg,avg_speed_kmph,stop_count,anomaly_flag,geo_trace,created_at
0,T000001,A000417,d0199,v0038,R000359,2025-11-18T16:00:00,2025-11-18T17:56:00,35.22,116,39.29,34.43,7.160,13.604,18.22,76,False,[],2025-11-13T11:09:24.181002
1,T000002,A000399,d0129,v0128,R000295,2025-11-18T11:30:00,2025-11-18T12:41:00,25.75,71,46.34,11.70,2.992,5.685,21.76,11,False,[],2025-11-13T11:09:24.182464
2,T000003,A000241,d0120,v0051,R000643,2025-11-16T20:00:00,2025-11-16T21:00:00,15.71,60,21.32,22.13,1.298,2.998,15.71,39,False,[],2025-11-13T11:09:24.183849
3,T000004,A000303,d0012,v0045,R000747,2025-11-17T11:30:00,2025-11-17T12:05:00,22.66,35,24.28,24.19,5.501,12.707,38.85,24,False,[],2025-11-13T11:09:24.185314
4,T000005,A000278,d0088,v0015,R001102,2025-11-17T05:15:00,2025-11-17T05:21:00,2.76,6,47.43,17.25,0.481,1.111,27.60,8,False,[],2025-11-13T11:09:24.186693


In [4]:
trip_file.isnull().sum()

trip_id           0
assignment_id     0
driver_id         0
vehicle_id        0
route_id          0
start_time        0
end_time          0
distance_km       0
duration_min      0
fuel_before_l     0
fuel_after_l      0
fuel_used_l       0
co2_kg            0
avg_speed_kmph    0
stop_count        0
anomaly_flag      0
geo_trace         0
created_at        0
dtype: int64

In [5]:
vehicle_file=pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\vehicle_agent_india_full.csv")
vehicle_file.head()

,vehicle_id,vehicle_type,model,fuel_type,baseline_kmpl,actual_kmpl,engine_health,maintenance_score,load_capacity_kg,avg_load_kg,load_factor,tyre_health,age_years,vehicle_multiplier,emission_factor,city
0,v0001,van,Maruti Omni,cng,13.28,9.41,8,7,1000,489,0.49,0.81,2,1.193,1.90,Delhi
1,v0002,cargo-auto,Bajaj Maxima,petrol,15.16,12.59,9,7,700,433,0.62,0.69,11,1.210,2.31,Pune
2,v0003,pickup,Mahindra Bolero Pickup,petrol,18.02,17.28,10,10,1200,493,0.41,0.77,5,1.123,2.31,Delhi
3,v0004,mini-truck,Ashok Leyland Dost,petrol,12.91,11.30,9,6,1200,1115,0.93,0.80,9,1.305,2.31,Delhi
4,v0005,truck,Eicher Pro 2049,diesel,14.78,13.18,10,9,700,586,0.84,0.90,9,1.252,2.68,Chennai


In [6]:
vehicle_file.isnull().sum()

vehicle_id            0
vehicle_type          0
model                 0
fuel_type             0
baseline_kmpl         0
actual_kmpl           0
engine_health         0
maintenance_score     0
load_capacity_kg      0
avg_load_kg           0
load_factor           0
tyre_health           0
age_years             0
vehicle_multiplier    0
emission_factor       0
city                  0
dtype: int64

In [13]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import folium
import random
from shapely.geometry import Point, LineString


In [14]:
"""
agents/autonomous_route_agent.py

Autonomous Route Agent (GIS-capable)
- Reads trips_india.csv and vehicle_agent_india_full.csv
- Plans eco-optimized route using OSMnx/NetworkX if available (fallback: simulated grid)
- Monitors simulated traffic and performs reroutes automatically
- Logs events to event_log_india.csv and notifies via a notification hook
- Usage:
    agent = AutonomousRouteAgent(trips_csv, vehicles_csv, event_log_csv="data/event_log_india.csv")
    agent.start_autonomous_trip_monitor("T000001", monitor_interval_s=10)
"""

import os
import time
import json
import logging
import random
from datetime import datetime
from typing import Dict, Any, List, Tuple, Optional

import pandas as pd
import numpy as np

# Try to import GIS libs; if unavailable, the agent will use a simulated grid graph
try:
    import osmnx as ox
    import networkx as nx
    from shapely.geometry import LineString, Point
    GIS_AVAILABLE = True
except Exception:
    import networkx as nx
    from shapely.geometry import LineString, Point
    GIS_AVAILABLE = False

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')


class AutonomousRouteAgent:
    def __init__(
        self,
        trips_csv: str,
        vehicles_csv: str,
        event_log_csv: str = "data/event_log_india.csv",
        default_place: str = "India",
        graph_bbox: Optional[Dict[str, float]] = None,
    ):
        """
        Initialize the autonomous route agent.

        trips_csv: path to trips_india.csv
        vehicles_csv: path to vehicle_agent_india_full.csv
        event_log_csv: path where event logs will be appended
        default_place: place string for OSMnx graph if GIS is available
        graph_bbox: optional dict {'north','south','east','west'} to limit OSMnx graph
        """
        self.trips_csv = trips_csv
        self.vehicles_csv = vehicles_csv
        self.event_log_csv = event_log_csv
        self.default_place = default_place
        self.graph_bbox = graph_bbox

        # load dataframes
        self.trips_df = pd.read_csv(trips_csv)
        self.vehicles_df = pd.read_csv(vehicles_csv)

        # ensure event log folder exists
        os.makedirs(os.path.dirname(event_log_csv) or ".", exist_ok=True)

        # build or simulate graph
        self.G = None
        self._build_graph()

        # internal state
        self.current_trip_state: Dict[str, Any] = {}

    # --------------------------
    # Graph building / helpers
    # --------------------------
    def _build_graph(self):
        """Attempt OSMnx graph, else build simulated grid graph."""
        if GIS_AVAILABLE:
            try:
                logging.info("Building OSM graph for '%s' ...", self.default_place)
                if self.graph_bbox:
                    b = self.graph_bbox
                    self.G = ox.graph_from_bbox(b["north"], b["south"], b["east"], b["west"], network_type="drive")
                else:
                    self.G = ox.graph_from_place(self.default_place, network_type="drive")
                logging.info("OSM Graph ready: nodes=%d edges=%d", len(self.G.nodes), len(self.G.edges))
                return
            except Exception as e:
                logging.warning("OSM graph build failed (%s). Falling back to simulated graph.", e)

        # fallback: build simulated grid graph (small)
        logging.info("Building simulated grid graph (fallback).")
        self.G = nx.DiGraph()
        grid_n = 6
        center_lat, center_lon = 20.5937, 78.9629  # approx center of India
        spacing_deg = 0.04  # approx ~4-5 km

        node_id = 0
        for i in range(grid_n):
            for j in range(grid_n):
                lat = center_lat + (i - grid_n/2) * spacing_deg
                lon = center_lon + (j - grid_n/2) * spacing_deg
                self.G.add_node(node_id, x=lon, y=lat)
                node_id += 1
        # connect 4-neighbors
        def nid(i, j): return i * grid_n + j
        for i in range(grid_n):
            for j in range(grid_n):
                u = nid(i, j)
                for di, dj in [(1,0),(-1,0),(0,1),(0,-1)]:
                    ni, nj = i+di, j+dj
                    if 0 <= ni < grid_n and 0 <= nj < grid_n:
                        v = nid(ni, nj)
                        dx = (self.G.nodes[u]['x'] - self.G.nodes[v]['x']) * 111
                        dy = (self.G.nodes[u]['y'] - self.G.nodes[v]['y']) * 111
                        dist_km = (dx*dx + dy*dy)**0.5
                        self.G.add_edge(u, v, length=dist_km * 1000.0)  # meters
        logging.info("Simulated graph built: nodes=%d edges=%d", len(self.G.nodes), len(self.G.edges))

    def _nearest_node(self, lat: float, lon: float):
        """Find nearest node in the graph to (lat, lon). Works for OSMnx and simulated graph."""
        best = None
        best_dist = float('inf')
        for n, d in self.G.nodes(data=True):
            nx_lon = d.get('x', d.get('lon', 0))
            nx_lat = d.get('y', d.get('lat', 0))
            dx = (nx_lon - lon) * 111
            dy = (nx_lat - lat) * 111
            dist = (dx*dx + dy*dy)**0.5
            if dist < best_dist:
                best_dist = dist
                best = n
        return best

    def _nodes_to_linestring(self, nodes: List[int]):
        coords = []
        for n in nodes:
            d = self.G.nodes[n]
            coords.append((d.get('x', d.get('lon', 0)), d.get('y', d.get('lat', 0))))
        try:
            return LineString(coords)
        except Exception:
            return None

    # --------------------------
    # Route planning
    # --------------------------
    def plan_eco_route_for_trip(self, trip_id: str, origin_override: Optional[Tuple[float, float]] = None,
                                dest_override: Optional[Tuple[float, float]] = None) -> Dict[str, Any]:
        """
        Plan an eco-optimized route for a given trip_id.
        If trip row does not contain coords, you can pass origin_override/dest_override as (lat,lon).
        """
        # get trip row
        if 'trip_id' not in self.trips_df.columns:
            raise ValueError("trips CSV must contain 'trip_id' column")
        rows = self.trips_df[self.trips_df['trip_id'] == trip_id]
        if rows.empty:
            raise ValueError(f"trip_id {trip_id} not found")
        trip = rows.iloc[0].to_dict()

        # Extract origin/dest coords
        origin_coords = None
        dest_coords = None
        # check explicit columns
        if 'origin_lat' in self.trips_df.columns and 'origin_lon' in self.trips_df.columns \
           and not pd.isna(trip.get('origin_lat')) and not pd.isna(trip.get('origin_lon')):
            origin_coords = (float(trip['origin_lat']), float(trip['origin_lon']))
        if 'dest_lat' in self.trips_df.columns and 'dest_lon' in self.trips_df.columns \
           and not pd.isna(trip.get('dest_lat')) and not pd.isna(trip.get('dest_lon')):
            dest_coords = (float(trip['dest_lat']), float(trip['dest_lon']))

        # geo_trace fallback
        if origin_coords is None and 'geo_trace' in trip and isinstance(trip['geo_trace'], str) and trip['geo_trace'] not in ('', '[]'):
            try:
                pts = json.loads(trip['geo_trace'])
                if pts and isinstance(pts, list):
                    origin_coords = tuple(pts[0])
                    dest_coords = tuple(pts[-1])
            except Exception:
                origin_coords = origin_coords

        # Apply overrides if provided
        if origin_override:
            origin_coords = origin_override
        if dest_override:
            dest_coords = dest_override

        # If still None, try to geocode textual city columns if present
        if origin_coords is None and 'origin' in trip and GIS_AVAILABLE and not pd.isna(trip['origin']):
            try:
                latlon = ox.geocode(f"{trip['origin']}, India")
                if latlon:
                    origin_coords = (latlon[0], latlon[1])
            except Exception:
                origin_coords = None

        if dest_coords is None and 'destination' in trip and GIS_AVAILABLE and not pd.isna(trip['destination']):
            try:
                latlon = ox.geocode(f"{trip['destination']}, India")
                if latlon:
                    dest_coords = (latlon[0], latlon[1])
            except Exception:
                dest_coords = None

        # As last fallback, use simulated nearest nodes
        if origin_coords is None or dest_coords is None:
            logging.info("Origin/Destination coords not found for trip %s — using simulated graph endpoints.", trip_id)
            origin_node = list(self.G.nodes())[0]
            dest_node = list(self.G.nodes())[-1]
        else:
            origin_node = self._nearest_node(origin_coords[0], origin_coords[1])
            dest_node = self._nearest_node(dest_coords[0], dest_coords[1])

        # get vehicle info
        vehicle_id = trip.get('vehicle_id') or trip.get('vehicle')
        vrows = self.vehicles_df[self.vehicles_df['vehicle_id'] == vehicle_id]
        if vrows.empty:
            logging.warning("Vehicle %s not found. Using default vehicle parameters.", vehicle_id)
            vehicle = {
                'vehicle_id': 'default',
                'actual_kmpl': 12.0,
                'vehicle_multiplier': 1.0,
                'emission_factor': 2.31
            }
        else:
            v = vrows.iloc[0]
            vehicle = {
                'vehicle_id': v.get('vehicle_id'),
                'actual_kmpl': v.get('actual_kmpl', v.get('baseline_kmpl', 12.0)),
                'vehicle_multiplier': v.get('vehicle_multiplier', 1.0),
                'emission_factor': v.get('emission_factor', 2.31)
            }

        # Set per-edge fuel_cost weights
        for u, v, data in self.G.edges(data=True):
            length_m = data.get('length', 100.0)
            length_km = length_m / 1000.0
            # avoid division by zero
            kmpl = max(0.001, float(vehicle['actual_kmpl']))
            data['fuel_cost'] = (length_km / kmpl) * float(vehicle['vehicle_multiplier'])
            data['length_km'] = length_km

        # compute eco path
        try:
            path_nodes = nx.shortest_path(self.G, origin_node, dest_node, weight='fuel_cost')
            edges = list(zip(path_nodes[:-1], path_nodes[1:]))
            total_km = sum(self.G[u][v]['length_km'] for u, v in edges)
            fuel_l = total_km / max(0.001, vehicle['actual_kmpl'])
            co2_kg = fuel_l * vehicle['emission_factor']
            route_geom = self._nodes_to_linestring(path_nodes)
        except Exception as e:
            logging.error("Path computation error: %s. Falling back to hops path.", e)
            path_nodes = nx.shortest_path(self.G, origin_node, dest_node)
            edges = list(zip(path_nodes[:-1], path_nodes[1:]))
            total_km = sum(self.G[u][v].get('length_km', 1.0) for u, v in edges)
            fuel_l = total_km / max(0.001, vehicle['actual_kmpl'])
            co2_kg = fuel_l * vehicle['emission_factor']
            route_geom = self._nodes_to_linestring(path_nodes)

        result = {
            'trip_id': trip_id,
            'origin_node': origin_node,
            'dest_node': dest_node,
            'route_nodes': path_nodes,
            'eco_distance_km': round(float(total_km), 3),
            'fuel_predicted_l': round(float(fuel_l), 3),
            'co2_emission_kg': round(float(co2_kg), 3),
            'route_geometry_wkt': route_geom.wkt if route_geom else None,
            'planned_at': datetime.utcnow().isoformat(),
            'reroute': False
        }

        # save internal state for this trip (so monitor can access)
        self.current_trip_state[trip_id] = {
            'trip': trip,
            'vehicle': vehicle,
            'path_nodes': path_nodes
        }

        # log event
        self._log_event({
            'event': 'planned_route',
            'trip_id': trip_id,
            'details': result,
            'time': datetime.utcnow().isoformat()
        })

        return result

    # --------------------------
    # Monitoring & rerouting (autonomous)
    # --------------------------
    def start_autonomous_trip_monitor(self, trip_id: str, monitor_interval_s: int = 15, max_iterations: Optional[int] = None):
        """
        Start autonomous monitoring loop for the given trip_id.
        - monitor_interval_s: seconds between checks (in real system would be more frequent)
        - max_iterations: optional limit for testing; None means run until KeyboardInterrupt
        """
        if trip_id not in self.current_trip_state:
            # plan route first
            logging.info("Planning route for trip %s before starting monitor...", trip_id)
            self.plan_eco_route_for_trip(trip_id)

        iteration = 0
        logging.info("Starting autonomous monitor for trip %s (interval=%ds). Press Ctrl+C to stop.", trip_id, monitor_interval_s)
        try:
            while True:
                iteration += 1
                # Stop condition for testing
                if max_iterations is not None and iteration > max_iterations:
                    logging.info("Max iterations reached (%d). Stopping monitor.", max_iterations)
                    break

                # perform a traffic check (this is simulated; replace with real traffic input)
                status = self._simulate_traffic_status()
                logging.info("Monitor iteration %d for trip %s -> traffic_status=%s", iteration, trip_id, status)

                if status == 'blocked':
                    logging.info("Traffic blocked detected -> attempt reroute for trip %s", trip_id)
                    new_result = self._attempt_reroute(trip_id)
                    if new_result:
                        logging.info("Reroute completed for trip %s. New distance=%.3f km, fuel=%.3f L",
                                     trip_id, new_result['eco_distance_km'], new_result['fuel_predicted_l'])
                elif status == 'moderate':
                    # optional: consider adjusting edge weights or notify
                    self._notify({'type': 'traffic_moderate', 'trip_id': trip_id, 'time': datetime.utcnow().isoformat()})
                else:
                    # clear: nothing to do
                    pass

                time.sleep(monitor_interval_s)
        except KeyboardInterrupt:
            logging.info("Monitor interrupted by user. Exiting monitor loop.")
        except Exception as e:
            logging.error("Error in monitor loop: %s", e)

    def _simulate_traffic_status(self) -> str:
        """Simulate traffic status. Probabilities: clear 75%, moderate 20%, blocked 5%"""
        return random.choices(['clear', 'moderate', 'blocked'], weights=[0.75, 0.20, 0.05])[0]

    def _attempt_reroute(self, trip_id: str) -> Optional[Dict[str, Any]]:
        """Perform reroute by making some edges highly undesirable and recomputing path."""
        state = self.current_trip_state.get(trip_id)
        if not state:
            logging.warning("No state found for trip %s; cannot reroute.", trip_id)
            return None
        path_nodes = state['path_nodes']
        if len(path_nodes) < 2:
            logging.warning("Path too short to reroute for trip %s", trip_id)
            return None

        # simulate blocking the middle edge of the current path
        mid_idx = len(path_nodes) // 2
        u = path_nodes[mid_idx - 1] if mid_idx - 1 >= 0 else path_nodes[0]
        v = path_nodes[mid_idx]
        if self.G.has_edge(u, v):
            logging.info("Simulating blockage on edge %s->%s", u, v)
            # mark edge as blocked by setting a very large fuel_cost
            self.G[u][v]['fuel_cost'] = 1e9

        # recompute path
        try:
            new_path = nx.shortest_path(self.G, path_nodes[0], path_nodes[-1], weight='fuel_cost')
            edges = list(zip(new_path[:-1], new_path[1:]))
            total_km = sum(self.G[u2][v2]['length_km'] for u2, v2 in edges)
            fuel_l = total_km / max(0.001, state['vehicle']['actual_kmpl'])
            co2_kg = fuel_l * state['vehicle']['emission_factor']
            route_geom = self._nodes_to_linestring(new_path)

            result = {
                'trip_id': trip_id,
                'route_nodes': new_path,
                'eco_distance_km': round(float(total_km), 3),
                'fuel_predicted_l': round(float(fuel_l), 3),
                'co2_emission_kg': round(float(co2_kg), 3),
                'route_geometry_wkt': route_geom.wkt if route_geom else None,
                'reroute': True,
                'reroute_time': datetime.utcnow().isoformat()
            }

            # update internal state
            state['path_nodes'] = new_path
            self.current_trip_state[trip_id] = state

            # log and notify
            self._log_event({
                'event': 'reroute',
                'trip_id': trip_id,
                'details': result,
                'time': datetime.utcnow().isoformat()
            })
            self._notify({'type': 'reroute', 'trip_id': trip_id, 'details': result})
            return result
        except Exception as e:
            logging.error("Reroute failed: %s", e)
            return None

    # --------------------------
    # Logging & notifications
    # --------------------------
    def _log_event(self, entry: Dict[str, Any]):
        """Append event row to event_log CSV (time, event, trip_id, details json)."""
        row = {
            'time_utc': datetime.utcnow().isoformat(),
            'event': entry.get('event'),
            'trip_id': entry.get('trip_id'),
            'details': json.dumps(entry.get('details', {}))
        }
        df_row = pd.DataFrame([row])
        if not os.path.exists(self.event_log_csv):
            df_row.to_csv(self.event_log_csv, index=False)
        else:
            df_row.to_csv(self.event_log_csv, mode='a', header=False, index=False)
        logging.info("Event logged: %s for trip %s", entry.get('event'), entry.get('trip_id'))

    def _notify(self, message: Dict[str, Any]):
        """Placeholder notification. Replace with integration to Notification Agent or push system."""
        logging.info("NOTIFY -> %s", json.dumps(message))

# --------------------------
# Demo / CLI usage
# --------------------------
if __name__ == "__main__":
    # change these paths to your local CSV paths
    TRIPS_CSV = r"C:\Users\ASUS\My_Learning\sat hack\trips_india.csv"
    VEHICLES_CSV = r"C:\Users\ASUS\My_Learning\sat hack\vehicle_agent_india_full.csv"
    EVENT_LOG = r"C:\Users\ASUS\My_Learning\sat hack\event_log_india.csv"

    # instantiate agent
    agent = AutonomousRouteAgent(TRIPS_CSV, VEHICLES_CSV, event_log_csv=EVENT_LOG, default_place="Chennai, India")

    # choose a trip from CSV to run autonomously
    # You can pick any trip_id from trips_india.csv (e.g., "T000001")
    trip_ids = list(agent.trips_df['trip_id'].astype(str).values)
    if not trip_ids:
        raise SystemExit("No trips found in trips CSV.")
    selected_trip = trip_ids[0]
    print("Starting autonomous monitoring for trip:", selected_trip)

    # plan initial route
    result = agent.plan_eco_route_for_trip(selected_trip)
    print("Initial route planned:", result)

    # start autonomous monitor loop (set max_iterations for demo if you want)
    # set max_iterations=10 for a short demo run, set to None for continuous until Ctrl+C
    agent.start_autonomous_trip_monitor(selected_trip, monitor_interval_s=8, max_iterations=12)


2025-11-13 19:12:07,724 INFO: Building OSM graph for 'Chennai, India' ...
2025-11-13 19:12:56,357 INFO: OSM Graph ready: nodes=68415 edges=173655
2025-11-13 19:12:56,360 INFO: Origin/Destination coords not found for trip T000001 — using simulated graph endpoints.
2025-11-13 19:12:56,553 ERROR: Path computation error: 'length_km'. Falling back to hops path.


Starting autonomous monitoring for trip: T000001


2025-11-13 19:12:56,564 INFO: Event logged: planned_route for trip T000001
2025-11-13 19:12:56,564 INFO: Starting autonomous monitor for trip T000001 (interval=8s). Press Ctrl+C to stop.
2025-11-13 19:12:56,565 INFO: Monitor iteration 1 for trip T000001 -> traffic_status=clear


Initial route planned: {'trip_id': 'T000001', 'origin_node': 30037235, 'dest_node': 13300900285, 'route_nodes': [30037235, 2197652486, 2197652485, 304918757, 304918752, 304918740, 2213431528, 2486858896, 11495672528, 11495672537, 2197639805, 10945638479, 10945638489, 2197639733, 2198125458, 2197639718, 317231282, 2197639707, 248468931, 6598418479, 6598418481, 248467469, 2482057562, 2482057558, 6776142294, 10970794270, 6965532360, 2482057541, 11392507518, 5458564176, 309179671, 11160949572, 1915751711, 11877912297, 309196383, 309196356, 309196369, 1682708500, 302897298, 302897389, 302897377, 11877907066, 259686460, 302897360, 302897388, 243048905, 247458625, 243048096, 1685058638, 1685049682, 247458700, 1685049659, 1685049658, 249798501, 259431126, 1877115132, 243061237, 249798504, 249798508, 259431028, 259431027, 249798535, 243061452, 407982148, 407982267, 407982147, 12429439969, 12429439974, 12527112718, 13300900285], 'eco_distance_km': 69.0, 'fuel_predicted_l': 8.234, 'co2_emission_k

2025-11-13 19:13:04,577 INFO: Monitor iteration 2 for trip T000001 -> traffic_status=clear
2025-11-13 19:13:12,582 INFO: Monitor iteration 3 for trip T000001 -> traffic_status=clear
2025-11-13 19:13:20,585 INFO: Monitor iteration 4 for trip T000001 -> traffic_status=clear
2025-11-13 19:13:28,597 INFO: Monitor iteration 5 for trip T000001 -> traffic_status=moderate
2025-11-13 19:13:28,598 INFO: NOTIFY -> {"type": "traffic_moderate", "trip_id": "T000001", "time": "2025-11-13T13:43:28.598314"}
2025-11-13 19:13:36,608 INFO: Monitor iteration 6 for trip T000001 -> traffic_status=moderate
2025-11-13 19:13:36,609 INFO: NOTIFY -> {"type": "traffic_moderate", "trip_id": "T000001", "time": "2025-11-13T13:43:36.609167"}
2025-11-13 19:13:44,627 INFO: Monitor iteration 7 for trip T000001 -> traffic_status=clear
2025-11-13 19:13:52,638 INFO: Monitor iteration 8 for trip T000001 -> traffic_status=clear
2025-11-13 19:14:00,642 INFO: Monitor iteration 9 for trip T000001 -> traffic_status=blocked
2025-1

In [15]:
import osmnx as ox
import networkx as nx
import pandas as pd
import folium
import random
import time
from shapely.geometry import LineString

# ==========================
# 1️⃣ Load Datasets
# ==========================
trip_file = pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\trips_india.csv")
vehicle_file = pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\vehicle_agent_india_full.csv")

# Example mapping of cities to lat/lon (you can extend this)
city_coords = {
    "Chennai": (13.0827, 80.2707),
    "Coimbatore": (11.0168, 76.9558),
    "Pune": (18.5204, 73.8567),
    "Delhi": (28.6139, 77.2090),
    "Mumbai": (19.0760, 72.8777)
}

# Pick one sample trip and vehicle
trip = trip_file.iloc[0]
vehicle = vehicle_file[vehicle_file["vehicle_id"] == trip["vehicle_id"]].iloc[0]

origin_city = vehicle["city"]
destination_city = "Coimbatore"  # Example destination
origin_point = city_coords.get(origin_city)
destination_point = city_coords.get(destination_city)

print(f"Starting from {origin_city} -> Destination: {destination_city}")
print(f"Vehicle Type: {vehicle['vehicle_type']} | Efficiency: {vehicle['actual_kmpl']} km/L")

# ==========================
# 2️⃣ Build GIS Graph
# ==========================
print("\nBuilding road network, please wait...")
G = ox.graph_from_place([origin_city, destination_city], network_type='drive')
orig_node = ox.distance.nearest_nodes(G, origin_point[1], origin_point[0])
dest_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# ==========================
# 3️⃣ Compute Eco Weight for Each Edge
# ==========================
fuel_eff = vehicle["actual_kmpl"]
emission_factor = vehicle["emission_factor"]

for u, v, data in G.edges(data=True):
    distance_km = data.get("length", 1) / 1000
    data["eco_weight"] = distance_km * emission_factor / max(fuel_eff, 0.1)

# ==========================
# 4️⃣ Compute Best Eco Route
# ==========================
eco_route = nx.shortest_path(G, orig_node, dest_node, weight="eco_weight")
eco_length = nx.shortest_path_length(G, orig_node, dest_node, weight="eco_weight")
eco_distance_km = sum(G[u][v][0].get("length", 1) for u, v in zip(eco_route[:-1], eco_route[1:])) / 1000

print(f"\n✅ Initial Eco Route Found!  Distance: {eco_distance_km:.2f} km | Eco Cost: {eco_length:.3f}")

# ==========================
# 5️⃣ Visualize Route
# ==========================
nodes, edges = ox.graph_to_gdfs(G)
route_geom = LineString([ (nodes.loc[node].x, nodes.loc[node].y) for node in eco_route ])
m = folium.Map(location=origin_point, zoom_start=7)
folium.GeoJson(route_geom, name="Eco Route", tooltip="Initial Route").add_to(m)
folium.Marker(origin_point, tooltip="Origin", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination_point, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)
m.save("eco_route_map.html")
print("🗺️ Route map saved as 'eco_route_map.html'")

# ==========================
# 6️⃣ Autonomous Traffic Monitoring
# ==========================
def simulate_traffic_condition():
    """Simulate traffic or blockage events randomly."""
    events = ["clear", "moderate", "heavy", "blocked"]
    return random.choices(events, weights=[0.5, 0.25, 0.2, 0.05])[0]

def reroute_if_blocked(G, eco_route):
    """If route is blocked, find alternate eco route."""
    u, v = random.choice(list(zip(eco_route[:-1], eco_route[1:])))
    print(f"\n🚧 Traffic blockage detected between {u}->{v}. Recomputing route...")
    if G.has_edge(u, v):
        # Make this edge very costly
        for key in G[u][v]:
            G[u][v][key]["eco_weight"] *= 999
    new_route = nx.shortest_path(G, orig_node, dest_node, weight="eco_weight")
    return new_route

print("\n🤖 Autonomous Route Monitoring Started...")
for i in range(5):  # simulate 5 monitoring cycles
    time.sleep(2)
    traffic = simulate_traffic_condition()
    print(f"[Cycle {i+1}] Traffic Condition: {traffic}")

    if traffic in ["heavy", "blocked"]:
        eco_route = reroute_if_blocked(G, eco_route)
        eco_distance_km = sum(G[u][v][0].get("length", 1) for u, v in zip(eco_route[:-1], eco_route[1:])) / 1000
        print(f"🔁 Rerouted Eco Route Distance: {eco_distance_km:.2f} km")
    else:
        print("✅ Route remains optimal")

print("\n✅ Autonomous monitoring complete. Route agent finished operations.")


Starting from Delhi -> Destination: Coimbatore
Vehicle Type: cargo-auto | Efficiency: 8.38 km/L

Building road network, please wait...


c:\Users\ASUS\anaconda\envs\ML_Compiler\lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 49 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)



✅ Initial Eco Route Found!  Distance: 23.18 km | Eco Cost: 5.256
🗺️ Route map saved as 'eco_route_map.html'

🤖 Autonomous Route Monitoring Started...
[Cycle 1] Traffic Condition: clear
✅ Route remains optimal
[Cycle 2] Traffic Condition: moderate
✅ Route remains optimal
[Cycle 3] Traffic Condition: heavy

🚧 Traffic blockage detected between 267196357->6485009721. Recomputing route...
🔁 Rerouted Eco Route Distance: 23.50 km
[Cycle 4] Traffic Condition: clear
✅ Route remains optimal
[Cycle 5] Traffic Condition: clear
✅ Route remains optimal

✅ Autonomous monitoring complete. Route agent finished operations.


In [20]:

import pandas as pd
import numpy as np
from math import radians, cos, sin, sqrt, atan2
import folium

# ------------------------
# Haversine Distance
# ------------------------
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1r, lon1r, lat2r, lon2r = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2r - lat1r, lon2r - lon1r
    a = sin(dlat/2)**2 + cos(lat1r)*cos(lat2r)*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

# ------------------------
# City Coordinates
# ------------------------
city_coords = {
    "Delhi": (28.6139, 77.2090),
    "Coimbatore": (11.0168, 76.9558)
}

origin = city_coords["Delhi"]
destination = city_coords["Coimbatore"]

distance = haversine_km(origin[0], origin[1], destination[0], destination[1])
print(f"Approximate Delhi → Coimbatore distance (Haversine): {distance:.2f} km")

# ------------------------
# Folium Map
# ------------------------
m = folium.Map(location=[(origin[0]+destination[0])/2, (origin[1]+destination[1])/2], zoom_start=5)
folium.Marker(origin, tooltip="Delhi", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination, tooltip="Coimbatore", icon=folium.Icon(color="red")).add_to(m)
folium.PolyLine([origin, destination], color="blue", weight=3, opacity=0.7, tooltip=f"Distance ~{distance:.1f} km").add_to(m)

m.save("delhi_coimbatore_map.html")
print("Map saved as 'delhi_coimbatore_map.html'. Open in a browser to view.")


Approximate Delhi → Coimbatore distance (Haversine): 1956.89 km
Map saved as 'delhi_coimbatore_map.html'. Open in a browser to view.


In [17]:
folium.Marker(
    origin_point,
    tooltip=f"Origin: {origin_city}",
    popup=f"Vehicle: {vehicle['vehicle_type']}\nEfficiency: {vehicle['actual_kmpl']} km/L",
    icon=folium.Icon(color="green")
).add_to(m)

folium.Marker(
    destination_point,
    tooltip=f"Destination: {destination_city}",
    icon=folium.Icon(color="red")
).add_to(m)


In [ ]:
def draw_route_on_map(route_nodes, map_obj, color="blue"):
    from shapely.geometry import LineString
    nodes, _ = ox.graph_to_gdfs(G)
    route_geom = LineString([(nodes.loc[node].x, nodes.loc[node].y) for node in route_nodes])
    folium.GeoJson(route_geom, name="Eco Route", tooltip="Rerouted").add_to(map_obj)

# Example usage after rerouting:
draw_route_on_map(eco_route, m, color="orange")
m.save("eco_route_map.html")


True

In [21]:
import webbrowser
webbrowser.open('delhi_coimbatore_map.html')

True

In [22]:
import osmnx as ox
import networkx as nx
import pandas as pd
import folium
import random
import time
from shapely.geometry import LineString

# ==========================
# 1️⃣ Load Data
# ==========================
trip_file = pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\trips_india.csv")
vehicle_file = pd.read_csv(r"C:\Users\ASUS\My_Learning\sat hack\vehicle_agent_india_full.csv")

# Example city coordinates
city_coords = {
    "Delhi": (28.6139, 77.2090),
    "Coimbatore": (11.0168, 76.9558),
    "Mumbai": (19.0760, 72.8777),
    "Pune": (18.5204, 73.8567),
    "Chennai": (13.0827, 80.2707)
}

# Pick one trip
trip = trip_file.iloc[0]
vehicle = vehicle_file[vehicle_file["vehicle_id"] == trip["vehicle_id"]].iloc[0]

origin_city = vehicle["city"]
destination_city = "Coimbatore"  # example
origin_point = city_coords[origin_city]
destination_point = city_coords[destination_city]

print(f"Trip: {origin_city} -> {destination_city}")
print(f"Vehicle: {vehicle['vehicle_type']} | Efficiency: {vehicle['actual_kmpl']} km/L | Emission Factor: {vehicle['emission_factor']}")

# ==========================
# 2️⃣ Build GIS Graph
# ==========================
print("\nBuilding road network, this may take a few minutes...")
G = ox.graph_from_place([origin_city, destination_city], network_type="drive")

# Get nearest nodes
orig_node = ox.distance.nearest_nodes(G, origin_point[1], origin_point[0])
dest_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# ==========================
# 3️⃣ Assign Eco Weight (Fuel + CO2)
# ==========================
fuel_efficiency = vehicle["actual_kmpl"]
emission_factor = vehicle["emission_factor"]  # kg CO2 per L fuel

for u, v, key, data in G.edges(keys=True, data=True):
    distance_km = data.get("length", 1) / 1000  # meters → km
    fuel_cost = distance_km / max(fuel_efficiency, 0.1)
    co2_cost = fuel_cost * emission_factor
    # Combine fuel + CO2 as eco weight (can adjust weights)
    data["eco_weight"] = fuel_cost + co2_cost

# ==========================
# 4️⃣ Compute Best Eco Route
# ==========================
eco_route = nx.shortest_path(G, orig_node, dest_node, weight="eco_weight")
eco_distance = sum(G[u][v][0].get("length", 1) for u, v in zip(eco_route[:-1], eco_route[1:])) / 1000
eco_eco_cost = sum(G[u][v][0]["eco_weight"] for u, v in zip(eco_route[:-1], eco_route[1:]))

print(f"\n✅ Initial Eco Route found! Distance: {eco_distance:.1f} km | Eco Cost: {eco_eco_cost:.3f}")

# ==========================
# 5️⃣ Folium Map Visualization
# ==========================
nodes, edges = ox.graph_to_gdfs(G)
route_geom = LineString([(nodes.loc[node].x, nodes.loc[node].y) for node in eco_route])

m = folium.Map(location=[(origin_point[0]+destination_point[0])/2, 
                         (origin_point[1]+destination_point[1])/2], zoom_start=6)

# Route line
folium.GeoJson(route_geom, tooltip="Eco Route", style_function=lambda x: {"color":"blue","weight":4,"opacity":0.7}).add_to(m)

# Start & End markers
folium.Marker(origin_point, tooltip="Origin", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination_point, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)

# Save map
map_file = "gis_eco_route_map.html"
m.save(map_file)
print(f"🗺️ Map saved as '{map_file}'")

# ==========================
# 6️⃣ Autonomous Traffic Monitoring
# ==========================
def simulate_traffic():
    return random.choices(["clear", "moderate", "heavy", "blocked"], weights=[0.5,0.25,0.2,0.05])[0]

def reroute_on_blockage(G, current_route):
    # pick random edge as blocked
    u, v = random.choice(list(zip(current_route[:-1], current_route[1:])))
    print(f"🚧 Traffic blockage detected on edge {u}->{v}")
    if G.has_edge(u, v):
        for key in G[u][v]:
            G[u][v][key]["eco_weight"] *= 9999  # make it extremely costly
    new_route = nx.shortest_path(G, orig_node, dest_node, weight="eco_weight")
    return new_route

print("\n🤖 Starting autonomous monitoring...")
for i in range(5):
    time.sleep(2)
    traffic = simulate_traffic()
    print(f"[Cycle {i+1}] Traffic condition: {traffic}")
    if traffic in ["heavy", "blocked"]:
        eco_route = reroute_on_blockage(G, eco_route)
        eco_distance = sum(G[u][v][0].get("length",1) for u,v in zip(eco_route[:-1], eco_route[1:]))/1000
        print(f"🔁 Rerouted distance: {eco_distance:.1f} km")
    else:
        print("✅ Route remains optimal")

print("\n✅ Autonomous monitoring complete. Route agent finished operations.")


Trip: Delhi -> Coimbatore
Vehicle: cargo-auto | Efficiency: 8.38 km/L | Emission Factor: 1.9

Building road network, this may take a few minutes...


c:\Users\ASUS\anaconda\envs\ML_Compiler\lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 49 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)



✅ Initial Eco Route found! Distance: 23.2 km | Eco Cost: 8.022
🗺️ Map saved as 'gis_eco_route_map.html'

🤖 Starting autonomous monitoring...
[Cycle 1] Traffic condition: clear
✅ Route remains optimal
[Cycle 2] Traffic condition: clear
✅ Route remains optimal
[Cycle 3] Traffic condition: moderate
✅ Route remains optimal
[Cycle 4] Traffic condition: heavy
🚧 Traffic blockage detected on edge 9837915331->4171935426
🔁 Rerouted distance: 23.4 km
[Cycle 5] Traffic condition: blocked
🚧 Traffic blockage detected on edge 4218568048->10682412741
🔁 Rerouted distance: 24.0 km

✅ Autonomous monitoring complete. Route agent finished operations.
